In [ ]:
path_to_data = "/media/raph/Elements/ml1/churn/"
nrows_for_members = None
nrows_for_transactions = 100
nrows_for_train = None
nrows_for_train_v2 = None
nrows_for_transactions = None
nrows_for_transactions_v2 = 1000000
nrows_for_test = None
nrows_for_test_v2 = None

In [ ]:
train = pd.read_csv(path_to_data+"train_v2.csv", nrows=nrows_for_train_v2)
train = pd.concat((train, pd.read_csv(path_to_data+"train.csv", nrows=nrows_for_train)), axis=0, ignore_index=True).reset_index(drop=True)
train.head()

In [ ]:
train.describe()

In [ ]:
members = pd.read_csv(path_to_data + "members_v3.csv", nrows=nrows_for_members)
members.head()

In [ ]:
members.describe()

In [ ]:
training = pd.merge(left = train,right = members,how = 'left',on=['msno'])

# changing type to int and putting -1 for missing values
training['city'] = training.city.apply(lambda x: int(x) if pd.notnull(x) else -1)
training['registered_via'] = training.registered_via.apply(lambda x: int(x) if pd.notnull(x) else -1)
training['bd'] = training.bd.apply(lambda x: int(x) if pd.notnull(x) else -1)
training['bd'] = training.bd.apply(lambda x: x if (10<x<100) else -1)

genders_encoding = {'male': 0, 'female': 1}
training['gender'] = training.gender.apply(lambda x: genders_encoding[x] if pd.notnull(x) else -1)

# changing date formats
training['registration_init_time'] = training.registration_init_time.apply(lambda x: datetime.strptime(str(int(x)), "%Y%m%d").date() if pd.notnull(x) else "NAN")
training.head()

In [ ]:
training.describe()

In [ ]:
del train
del members

from tools import change_datatype, change_datatype_float
# reducing memory usage:
change_datatype(training)
change_datatype_float(training)

### Lets make a short explanation here.
We can see that we have a train dataset with 970m entries. And the members dataset has 6M entries. So we actually have a huge number of members and the train dataset is giving us a list of users with the description of whether they churned or not. The test dataset will give us another set of users asking us which will churn during the next month.

In [ ]:
#registration_init_time yearly trend
training['registration_init_time_year'] = pd.DatetimeIndex(training['registration_init_time']).year
training['registration_init_time_year'] = training.registration_init_time_year.apply(lambda x: int(x) if pd.notnull(x) else "NAN" )
year_count=training['registration_init_time_year'].value_counts()

plt.figure(figsize=(12,12))
plt.subplot(311)
year_order = training['registration_init_time_year'].unique()
year_order=sorted(year_order, key=lambda x: str(x))
year_order = sorted(year_order, key=lambda x: float(x))
sns.barplot(year_count.index, year_count.values,order=year_order)
plt.ylabel('Count', fontsize=12)
plt.xlabel('Year', fontsize=12)
plt.xticks(rotation='vertical')
plt.title("Yearly Trend of registration_init_time", fontsize=12)
plt.show()
year_count_2 = Counter(training['registration_init_time_year']).most_common();

### Lets see how the year of registration influences the probability of churn:

In [ ]:
churn_mean_on_year = []
for y in year_order:
    current = training.loc[training['registration_init_time_year']==y]['is_churn'].mean()
    churn_mean_on_year.append(current)

plt.figure(figsize=(12,12))
plt.subplot(311)
sns.barplot(year_order, churn_mean_on_year,order=year_order)
plt.ylabel('Churning ratio', fontsize=12)
plt.xlabel('Year', fontsize=12)
plt.xticks(rotation='vertical')
plt.title("Churning ratio vs registration_init_time", fontsize=12)
plt.show()

In [ ]:
#Gender count
plt.figure(figsize=(12,12))
plt.subplot(413)
sns.countplot(x="gender", data=training)
plt.ylabel('Count', fontsize=12)
plt.xlabel('Gender', fontsize=12)
plt.xticks(rotation='vertical')
plt.title("Gender Count", fontsize=12)
plt.show()
gender_count = Counter(training['gender']).most_common()

genders = []
churn_mean_on_gender = []
for (g, _) in gender_count:
    genders.append(g)
    churn_mean_on_gender.append(training.loc[training['gender']==g]['is_churn'].mean())

plt.figure(figsize=(12,12)) 
plt.subplot(413)
sns.barplot(genders, churn_mean_on_gender, genders)
plt.ylabel('Churning ratio', fontsize=12)
plt.xlabel('Gender', fontsize=12)
plt.xticks(rotation='vertical')
plt.title("Churning ratio vs gender", fontsize=12)
plt.show();

In [ ]:
# City count in Members Data Set
plt.figure(figsize=(12,12))
plt.subplot(311)
sns.countplot(x="city", data=training)
plt.ylabel('Count', fontsize=12)
plt.xlabel('City', fontsize=12)
plt.xticks(rotation='vertical')
plt.title("City count in Training dataset", fontsize=12)
plt.show()
city_count = Counter(training['city']).most_common()

#Registered Via Count in Members Data Set
plt.figure(figsize=(12,12))
plt.subplot(312)
sns.countplot(x="registered_via", data=training)
plt.ylabel('Count', fontsize=12)
plt.xlabel('Registered Via', fontsize=12)
plt.xticks(rotation='vertical')
plt.title("Registered Via Count in Training Data Set", fontsize=12)
plt.show()
RV_count = Counter(training['registered_via']).most_common();

In [ ]:
churn_mean_on_city = []
cities = []
for (city, count) in city_count:
    if count > 1000:
        current = training.loc[training['city']==city]['is_churn'].mean()
        churn_mean_on_city.append(current)
        cities.append(city)
        
plt.figure(figsize=(12,12)) 
plt.subplot(413)
sns.barplot(cities, churn_mean_on_city)
plt.ylabel('Churning ratio', fontsize=12)
plt.xlabel('City', fontsize=12)
plt.xticks(rotation='vertical')
plt.title("Churning ratio vs city", fontsize=12)
plt.show()

In [ ]:
churn_mean_on_rv = []
rv = []
for (source, count) in RV_count:
    if count > 1000:
        current = training.loc[training['registered_via']==source]['is_churn'].mean()
        churn_mean_on_rv.append(current)
        rv.append(source)
        
plt.figure(figsize=(12,12)) 
plt.subplot(413)
sns.barplot(rv, churn_mean_on_rv)
plt.ylabel('Churning ratio', fontsize=12)
plt.xlabel('Registered via', fontsize=12)
plt.xticks(rotation='vertical')
plt.title("Churning ratio vs Registered via ", fontsize=12)
plt.show()

Lets try to fit a simple model to this first set of data and see what kind of results we get ! (this will enable us to have a reference for any other models)

In [ ]:
training.head()

In [ ]:
import time
training['registration_init_time'] = training.registration_init_time.apply(lambda x: time.mktime(x.timetuple()) if not type(x)==type('str') else 0.0)
training.head()

In [ ]:
X = training.iloc[:, [2, 3, 4, 5, 6]].values
y = training.iloc[:, 1].values

# Splitting the dataset into the Training set and Test set
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

print("fitting model...")
# Fitting Random Forest Classification to the Training set
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)

print("predicting...")
# Predicting the Test set results
y_pred = classifier.predict_proba(X_test)[:, 1]
y_pred_train = classifier.predict_proba(X_train)[:, 1]

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm_train = confusion_matrix(y_train, y_pred_train>0.5)
print("\nconfusion matrix on train: ")
print(cm_train)
cm_test = confusion_matrix(y_test, y_pred>0.5)
print("\nconfusion matrix on test: ")
print(cm_test)

from tools import log_loss
print("\nloss on train: ")
print(log_loss(y_train, y_pred_train))
print("\nloss on test: ")
print(log_loss(y_test, y_pred))

### Now lets explore the transactions dataset:

In [ ]:
transactions = pd.read_csv(path_to_data+"transactions.csv", nrows=nrows_for_transactions)
transactions = pd.concat((transactions, pd.read_csv(path_to_data+"transactions_v2.csv", nrows=nrows_for_transactions_v2)), axis=0, ignore_index=True).reset_index(drop=True)
transactions['transaction_date'] = transactions.transaction_date.apply(lambda x: datetime.strptime(str(int(x)), "%Y%m%d").date() if pd.notnull(x) else "NAN")
transactions['membership_expire_date'] = transactions.membership_expire_date.apply(lambda x: datetime.strptime(str(int(x)), "%Y%m%d").date() if pd.notnull(x) else "NAN")
transactions['payment_method_id'] = transactions.payment_method_id.apply(lambda x: int(x) if pd.notnull(x) else -1)
transactions.head()

In [ ]:
change_datatype(transactions)
change_datatype_float(transactions)
transactions.describe()

In [ ]:
print("computing")
user_count = Counter(transactions['msno']).most_common()

In [ ]:
training.head()

In [ ]:
print("hello")
training["number_of_transactions"] = 0
training.set_index('msno', inplace=True)
training.head()

In [ ]:
counts = []
i = 0
print("helo")
for id_, count in user_count:
    counts.append(count)
    if i % 100000 == 0:
        print(i)
    i+=1
    try:
        training['number_of_transactions'].loc[id_] = count
    except:
        pass
    
bins = range(1, 5)

sns.distplot(counts[5000:],kde=False, norm_hist=True, bins=bins)
plt.ylabel('Percentage', fontsize=12)
plt.xlabel('Number of transactions', fontsize=12)
plt.xticks(rotation='vertical')
plt.title("Number of transactions for the same user", fontsize=12)
plt.show()

### Lets comment this plot:
So there is in most cases a single transaction in the dataset for each user. This means that it might not be very helpful to look at the evolution of transactions for a user. What we might want to do is to keep only one transaction for each user and we are going to keep only the earliest transaction.

In [ ]:
# number of transactions vs churn ratio
i = 0
churn_elements = np.array([0, 0, 0, 0])
total_elements = np.array([0, 0, 0, 0])
i = 0

for element in training.itertuples():
    if i % 100000 == 0:
        print(i)
    i+=1
    if element.number_of_transactions < 4:
        total_elements[element.number_of_transactions] += 1
        if element.is_churn:
            churn_elements[element.number_of_transactions] += 1

print(churn_elements)
print(total_elements)

for i in range (len(churn_elements)):
    churn_elements[i] /= total_elements[i]
    


In [ ]:
transactions['transaction_date_month'] = pd.DatetimeIndex(transactions['transaction_date']).month
transactions['transaction_date_month'] = transactions.transaction_date_month.apply(lambda x: int(x) if pd.notnull(x) else -1)
month_count=transactions['transaction_date_month'].value_counts()

plt.figure(figsize=(12,4))
month_order = transactions['transaction_date_month'].unique()
month_order=sorted(month_order, key=lambda x: str(x))
month_order = sorted(month_order, key=lambda x: float(x))
sns.barplot(month_count.index, month_count.values,order=month_order)
plt.ylabel('Count', fontsize=12)
plt.xlabel('Month', fontsize=12)
plt.title("Monthly Trend of transaction_date_month", fontsize=12)
plt.show()
year_count_2 = Counter(transactions['transaction_date_month']).most_common();

In [ ]:
transactions['membership_expire_date_month'] = pd.DatetimeIndex(transactions['membership_expire_date']).month
transactions['membership_expire_date_month'] = transactions.membership_expire_date_month.apply(lambda x: int(x) if pd.notnull(x) else -1)
month_count=transactions['membership_expire_date_month'].value_counts()

plt.figure(figsize=(12,4))
month_order = transactions['membership_expire_date_month'].unique()
month_order=sorted(month_order, key=lambda x: str(x))
month_order = sorted(month_order, key=lambda x: float(x))
sns.barplot(month_count.index, month_count.values,order=month_order)
plt.ylabel('Count', fontsize=12)
plt.xlabel('Month', fontsize=12)
plt.title("Monthly Trend of membership_expire_date_month", fontsize=12)
plt.show()
year_count_2 = Counter(transactions['membership_expire_date_month']).most_common()

In [ ]:
# conversion en set augmente la vitesse de façon PHENOMENALE
id_set = set(training['msno'].values)
bins = np.array([1 if id_ in id_set else 0 for id_ in transactions['msno']])
# quel pourcentage des transactions fait partie du training ?
print(np.mean(bins))

### Il est temps de loader la base de test

In [ ]:
test = pd.read_csv(path_to_data+"sample_submission_v2.csv")
test.head()

In [ ]:
test.describe()

In [ ]:
id_set = set(test['msno'].values)
bins = np.array([1 if id_ in id_set else 0 for id_ in transactions['msno']])
# quel pourcentage des transactions fait partie du test ?
print(np.mean(bins))

In [ ]:
id_set = set(training['msno'].values)
bins = np.array([1 if id_ in id_set else 0 for id_ in test['msno']])
# quel pourcentage des données de test sont dans le fichier de train ?
print(np.mean(bins))

Donc 88% des données de test sont dans le fichier de train. C'est très bizarre du coup. Mais bon comme on essaye de prédire un évènement nouveau ça peut avoir du sens. Ce que je comprends pas c'est surtout pourquoi il y a si peu de transactions... En moyenne les gens reprennent l'abonnement pour deux mois donc ça peut expliquer qu'il y ait moins de transactions.

un kernel pas mal:
https://www.kaggle.com/the1owl/regressing-during-insomnia-0-21496
Il fait un apprentissage assez bourin et il arrive à un score pas trop mal. Ce qui est intéressant c'est surtout les features qu'il utilise.

Un autre kernel qui fait un peu de memory reduction ce qui pourrait faire du bien:
https://www.kaggle.com/jeru666/did-you-think-of-these-features
